In [13]:
import numpy as np 
import matplotlib.pyplot as plt 

arr = np.load()
x = np.arange(len(arr))

plt.scatter(x,arr)
plt.show

Mismatch by input state:
	000000:  9.99e-01 (1.0)
	000001:  9.52e-01 (1.0)
	000010:  9.71e-01 (1.0)
	000011:  9.99e-01 (1.0)
	000100:  9.79e-01 (-1.0)
	000101:  9.99e-01 (1.0)
	000110:  9.59e-01 (-1.0)
	000111:  9.95e-01 (1.0)
	001000:  4.46e-01 (1.0)
	001001:  4.54e-01 (1.0)
	001010:  9.99e-01 (1.0)
	001011:  9.98e-01 (-1.0)
	001100:  9.99e-01 (-1.0)
	001101:  1.86e-01 (1.0)
	001110:  1.89e-01 (1.0)
	001111:  1.84e-01 (1.0)
	010000:  1.94e-01 (1.0)
	010001:  9.97e-01 (-1.0)
	010010:  9.96e-01 (-1.0)
	010011:  9.96e-01 (-1.0)
	010100:  9.96e-01 (-1.0)
	010101:  9.96e-01 (-1.0)
	010110:  9.96e-01 (-1.0)
	010111:  9.96e-01 (-1.0)
	011000:  9.97e-01 (-1.0)
	011001:  9.97e-01 (-1.0)
	011010:  9.96e-01 (-1.0)
	011011:  9.96e-01 (-1.0)
	011100:  9.96e-01 (-1.0)
	011101:  9.97e-01 (-1.0)
	011110:  9.96e-01 (-1.0)
	011111:  9.96e-01 (-1.0)
	100000:  9.96e-01 (-1.0)
	100001:  9.96e-01 (-1.0)
	100010:  9.96e-01 (-1.0)
	100011:  9.96e-01 (-1.0)
	100100:  9.96e-01 (-1.0)
	100101:  9.96e-01 (-1.0)
